In [1]:
import datetime

import csv

import pandas as pd

from sqlalchemy import create_engine

import yfinance as yf 

from bs4 import BeautifulSoup

import requests

from pyspark.sql import SparkSession

import pyspark

from pyspark.context import SparkContext

import pyspark.sql.functions as fc

import pymysql

from pyspark.sql import Row

from pyspark.sql.types import *

import json

import numpy as np

In [2]:
sc = SparkContext.getOrCreate()

spark = SparkSession(sc)


In [3]:
stockspark = spark.read.json('stockdata.json')

companyspark = spark.read.json('companyinfo.json')


In [4]:
companyspark.show(10)


+--------------------+--------------------+------+
|                Name|              Sector|Symbol|
+--------------------+--------------------+------+
|Agilent Technolog...|         Health Care|     A|
|American Airlines...|         Industrials|   AAL|
|  Advance Auto Parts|Consumer Discreti...|   AAP|
|          Apple Inc.|Information Techn...|  AAPL|
|         AbbVie Inc.|         Health Care|  ABBV|
|AmerisourceBergen...|         Health Care|   ABC|
|         ABIOMED Inc|         Health Care|  ABMD|
| Abbott Laboratories|         Health Care|   ABT|
|       Accenture plc|Information Techn...|   ACN|
|          Adobe Inc.|Information Techn...|  ADBE|
+--------------------+--------------------+------+
only showing top 10 rows



In [5]:
stockspark.show(10)


+-----+----------+---------+-----+-----+----+-----+---------+
|close|      date|dividends| high|  low|name| open|   volume|
+-----+----------+---------+-----+-----+----+-----+---------+
| 85.4|01/02/2020|      0.0| 85.8|84.65|   A|85.35|1410500.0|
|84.03|01/03/2020|      0.0|84.78|83.96|   A|84.13|1118300.0|
|84.28|01/06/2020|      0.0|84.28|83.06|   A|83.46|1993200.0|
|84.53|01/07/2020|      0.0|84.71| 83.4|   A|83.42|1684700.0|
|85.37|01/08/2020|      0.0|85.91|84.65|   A|85.41|1847600.0|
|86.71|01/09/2020|      0.0|87.14|85.62|   A| 85.9|1912700.0|
|87.03|01/10/2020|      0.0|87.67|86.76|   A|87.16|1417000.0|
| 86.9|01/13/2020|      0.0|87.75|86.18|   A|87.25|1630200.0|
|87.43|01/14/2020|      0.0|87.64|86.14|   A|86.71|1675200.0|
|88.05|01/15/2020|      0.0|88.54|86.99|   A|87.07|1630400.0|
+-----+----------+---------+-----+-----+----+-----+---------+
only showing top 10 rows



In [6]:
sectorjoin=stockspark.join(companyspark,stockspark.name==companyspark.Symbol).select(stockspark.date,companyspark.Symbol,stockspark.open,stockspark.close,stockspark.high,stockspark.low,stockspark.volume,stockspark.dividends,companyspark.Sector)

In [7]:
sectorjoin.show(5)


+----------+------+-----+-----+-----+-----+---------+---------+-----------+
|      date|Symbol| open|close| high|  low|   volume|dividends|     Sector|
+----------+------+-----+-----+-----+-----+---------+---------+-----------+
|01/02/2020|     A|85.35| 85.4| 85.8|84.65|1410500.0|      0.0|Health Care|
|01/03/2020|     A|84.13|84.03|84.78|83.96|1118300.0|      0.0|Health Care|
|01/06/2020|     A|83.46|84.28|84.28|83.06|1993200.0|      0.0|Health Care|
|01/07/2020|     A|83.42|84.53|84.71| 83.4|1684700.0|      0.0|Health Care|
|01/08/2020|     A|85.41|85.37|85.91|84.65|1847600.0|      0.0|Health Care|
+----------+------+-----+-----+-----+-----+---------+---------+-----------+
only showing top 5 rows



In [8]:
sectorjoin[['Sector']].distinct().show()


+--------------------+
|              Sector|
+--------------------+
|         Health Care|
|Communication Ser...|
|              Energy|
|Information Techn...|
|         Real Estate|
|           Materials|
|Consumer Discreti...|
|           Utilities|
|    Consumer Staples|
|         Industrials|
|          Financials|
+--------------------+



In [15]:
ind=[('Industrials',),
 ('Health Care',),
 ('Information Technology',),
 ('Communication Services',),
 ('Consumer Discretionary',),
 ('Utilities',),
 ('Financials',),
 ('Materials',),
 ('Real Estate',),
 ('Consumer Staples',),
 ('Energy',)]

In [9]:
indu=sectorjoin[['Sector']].distinct()

In [10]:
#sectorjoin.groupBy(['Symbol','Sector','date']).agg({'close':'max'}).show()
sectorjoin.groupBy(['Symbol']).agg(fc.max('close').alias('2020HIGH')).show()


+------+--------+
|Symbol|2020HIGH|
+------+--------+
|  ALXN|  127.62|
|   GIS|    65.2|
|     K|   70.91|
|   LEN|   84.68|
|  SPGI|  378.41|
|  ABMD|   317.0|
|   AIV|   40.38|
|   AVY|  152.35|
|   RTX|     NaN|
|   MMM|  176.37|
|   PKI|   141.3|
|   PPG|  142.82|
|    RF|   16.48|
|   AXP|  134.99|
|   CDW|  143.95|
|    CI|  222.51|
|   IRM|   32.21|
|   WEC|     NaN|
|  INFO|   91.48|
|   PFG|   54.92|
+------+--------+
only showing top 20 rows



In [13]:

sectorjoin.groupBy(['Sector','Symbol']).agg(fc.max('close').alias('Period_High'),fc.min('close').alias('Period_Low')).orderBy('Sector').show()



+--------------------+------+-----------+----------+
|              Sector|Symbol|Period_High|Period_Low|
+--------------------+------+-----------+----------+
|Communication Ser...|  NWSA|       17.2|      8.13|
|Communication Ser...|  NFLX|     556.55|    298.84|
|Communication Ser...|  TWTR|      52.43|      22.0|
|Communication Ser...|   DIS|      148.2|     85.76|
|Communication Ser...| DISCK|       30.6|     16.46|
|Communication Ser...|  VIAC|      41.16|     11.19|
|Communication Ser...|  GOOG|    1763.37|   1056.62|
|Communication Ser...| GOOGL|     1762.5|   1054.13|
|Communication Ser...|  TTWO|     177.52|    100.15|
|Communication Ser...|   OMC|      77.85|      45.8|
|Communication Ser...|    VZ|       61.1|     48.35|
|Communication Ser...|   NWS|      17.24|      8.09|
|Communication Ser...| DISCA|      32.77|     18.36|
|Communication Ser...|  TMUS|     125.15|     74.32|
|Communication Ser...|  ATVI|      86.84|     51.71|
|Communication Ser...|   CTL|      14.17|     

In [ ]:
sectorjoin.groupBy(['Sector','Symbol']).agg(fc.max('close').alias('Period_High'),fc.min('close').alias('Period_Low')).orderBy('Sector').show()

In [21]:
health_care=sectorjoin[(sectorjoin.Sector=='Health Care')].groupBy(['Symbol','Sector']).agg(fc.max('close').alias('Period_High'),fc.min('close').alias('Period_Low'))
health_care.show(5)
information_technology=sectorjoin[(sectorjoin.Sector=='Information Technology')].groupBy(['Symbol','Sector']).agg(fc.max('close').alias('Period_High'),fc.min('close').alias('Period_Low'))
information_technology.show(5)
communication_services=sectorjoin[(sectorjoin.Sector=='Communication Services')].groupBy(['Symbol','Sector']).agg(fc.max('close').alias('Period_High'),fc.min('close').alias('Period_Low'))
communication_services.show(5)
consumer_discretionary=sectorjoin[(sectorjoin.Sector=='Consumer Discretionary')].groupBy(['Symbol','Sector']).agg(fc.max('close').alias('Period_High'),fc.min('close').alias('Period_Low'))
consumer_discretionary.show(5)
utilities=sectorjoin[(sectorjoin.Sector=='Utilities')].groupBy(['Symbol','Sector']).agg(fc.max('close').alias('Period_High'),fc.min('close').alias('Period_Low'))
utilities.show(5)
financials=sectorjoin[(sectorjoin.Sector=='Financials')].groupBy(['Symbol','Sector']).agg(fc.max('close').alias('Period_High'),fc.min('close').alias('Period_Low'))
financials.show(5)
materials=sectorjoin[(sectorjoin.Sector=='Materials')].groupBy(['Symbol','Sector']).agg(fc.max('close').alias('Period_High'),fc.min('close').alias('Period_Low'))
materials.show(5)
real_estate=sectorjoin[(sectorjoin.Sector=='Real Estate')].groupBy(['Symbol','Sector']).agg(fc.max('close').alias('Period_High'),fc.min('close').alias('Period_Low'))
real_estate.show(5)
consumer_staples=sectorjoin[(sectorjoin.Sector=='Consumer Staples')].groupBy(['Symbol','Sector']).agg(fc.max('close').alias('Period_High'),fc.min('close').alias('Period_Low'))
consumer_staples.show(5)
energy=sectorjoin[(sectorjoin.Sector=='Energy')].groupBy(['Symbol','Sector']).agg(fc.max('close').alias('Period_High'),fc.min('close').alias('Period_Low'))
energy.show(5)

+------+-----------+-----------+----------+
|Symbol|     Sector|Period_High|Period_Low|
+------+-----------+-----------+----------+
|  AMGN|Health Care|     259.21|    179.81|
|  INCY|Health Care|     109.69|     63.18|
|  REGN|Health Care|     658.21|    336.18|
|   MCK|Health Care|     183.44|    115.13|
|  ILMN|Health Care|     400.74|     209.2|
+------+-----------+-----------+----------+
only showing top 5 rows

+------+--------------------+-----------+----------+
|Symbol|              Sector|Period_High|Period_Low|
+------+--------------------+-----------+----------+
|  INTC|Information Techn...|      66.81|     43.74|
|   DXC|Information Techn...|      36.05|      9.13|
|   TXN|Information Techn...|     157.74|     91.48|
|   XRX|Information Techn...|       36.6|     14.43|
|  NVDA|Information Techn...|     582.48|    196.25|
+------+--------------------+-----------+----------+
only showing top 5 rows

+------+--------------------+-----------+----------+
|Symbol|              Se

In [38]:
sectorjoin[(sectorjoin.Sector=='Health Care')].groupBy(['Symbol','Sector']).agg(fc.max('close').alias('Period_High'),fc.min('close').alias('Period_Low')).show()



+------+-----------+-----------+----------+
|Symbol|     Sector|Period_High|Period_Low|
+------+-----------+-----------+----------+
|  AMGN|Health Care|     259.21|    179.81|
|  INCY|Health Care|     109.69|     63.18|
|  REGN|Health Care|     658.21|    336.18|
|   MCK|Health Care|     183.44|    115.13|
|  ILMN|Health Care|     400.74|     209.2|
|   CAH|Health Care|      58.65|      39.8|
|  PRGO|Health Care|      60.07|     40.32|
|  ALXN|Health Care|     127.62|     75.56|
|   BSX|Health Care|      45.71|     25.83|
|   STE|Health Care|     191.82|    107.23|
|   MDT|Health Care|     119.08|     71.59|
|   HUM|Health Care|     452.01|    213.29|
|  ABBV|Health Care|      99.48|     61.94|
|   SYK|Health Care|     234.35|    125.28|
|   WST|Health Care|     301.49|    128.69|
|    LH|Health Care|     214.42|    104.32|
|   MYL|Health Care|      22.85|     13.26|
|  HSIC|Health Care|      73.71|     43.18|
|    CI|Health Care|     222.51|    130.06|
|   ZTS|Health Care|     174.35|

In [57]:
n2.show(5)

+------+-----------+----------+----------+
|Symbol|Period_High|      date|Period_Low|
+------+-----------+----------+----------+
|   DGX|     129.37|08/03/2020|     72.08|
|   TFX|        NaN|11/12/2020|    224.67|
|   WST|     301.49|10/12/2020|    128.69|
|   ZTS|     174.35|11/04/2020|     92.27|
|  PRGO|      60.07|02/25/2020|     40.32|
+------+-----------+----------+----------+
only showing top 5 rows



In [142]:
n1=sectorjoin[(sectorjoin.Sector=='Energy')].groupBy(['Symbol','Sector']).agg(fc.max('close').alias('Period_High'),fc.min('close').alias('Period_Low'))

In [143]:
n2=n1.join(sectorjoin,sectorjoin.Symbol==n1.Symbol).filter(sectorjoin.close==n1.Period_High).select(sectorjoin.Sector,n1.Symbol,n1.Period_High,sectorjoin.date.alias('High_Date'),n1.Period_Low)

n3=n1.join(sectorjoin,sectorjoin.Symbol==n1.Symbol).filter(sectorjoin.close==n1.Period_Low).select(sectorjoin.Sector,n1.Symbol,n1.Period_High,n1.Period_Low,sectorjoin.date.alias('Low_Date'))

energytable=n2.join(n3,n3.Symbol==n2.Symbol).filter(n2.Period_High!= 'NaN').select(n2.Sector,n2.Symbol,n2.Period_High,n2.High_Date,n3.Period_Low,n3.Low_Date)

In [145]:
energytable.show(1)

+------+------+-----------+----------+----------+----------+
|Sector|Symbol|Period_High| High_Date|Period_Low|  Low_Date|
+------+------+-----------+----------+----------+----------+
|Energy|   OXY|       45.8|01/15/2020|      8.88|10/28/2020|
+------+------+-----------+----------+----------+----------+
only showing top 1 row



In [85]:
healthcaretable.show(5)

+-----------+------+-----------+----------+----------+----------+
|     Sector|Symbol|Period_High| High_Date|Period_Low|  Low_Date|
+-----------+------+-----------+----------+----------+----------+
|Health Care|  ALXN|     127.62|11/11/2020|     75.56|03/16/2020|
|Health Care|  ABMD|      317.0|08/05/2020|    130.51|03/20/2020|
|Health Care|   PKI|      141.3|11/06/2020|     66.57|03/16/2020|
|Health Care|    CI|     222.51|02/12/2020|    130.06|03/23/2020|
|Health Care|   STE|     191.82|10/12/2020|    107.23|03/23/2020|
+-----------+------+-----------+----------+----------+----------+
only showing top 5 rows



In [89]:
informationtechtable.show(5)

+--------------------+------+-----------+----------+----------+----------+
|              Sector|Symbol|Period_High| High_Date|Period_Low|  Low_Date|
+--------------------+------+-----------+----------+----------+----------+
|Information Techn...|   CDW|     143.95|11/09/2020|     79.01|03/20/2020|
|Information Techn...|  FLIR|      58.06|02/13/2020|     25.31|03/20/2020|
|Information Techn...|   FLT|     328.85|02/06/2020|    171.72|03/23/2020|
|Information Techn...|  AAPL|     133.95|09/01/2020|     55.29|03/23/2020|
|Information Techn...|  CSCO|      48.63|02/12/2020|     32.33|03/12/2020|
+--------------------+------+-----------+----------+----------+----------+
only showing top 5 rows



In [114]:
industrialstable.show(5)

+-----------+------+-----------+----------+----------+----------+
|     Sector|Symbol|Period_High| High_Date|Period_Low|  Low_Date|
+-----------+------+-----------+----------+----------+----------+
|Industrials|   MMM|     176.37|01/14/2020|    115.66|03/23/2020|
|Industrials|  INFO|      91.48|11/11/2020|     45.09|03/23/2020|
|Industrials|   SNA|     172.67|11/10/2020|     94.07|04/03/2020|
|Industrials|   FTV|      70.33|11/09/2020|     35.08|03/23/2020|
|Industrials|   ROP|     452.59|08/13/2020|    253.22|03/23/2020|
+-----------+------+-----------+----------+----------+----------+
only showing top 5 rows



In [95]:
communicationservicetable.show(5)

+--------------------+------+-----------+----------+----------+----------+
|              Sector|Symbol|Period_High| High_Date|Period_Low|  Low_Date|
+--------------------+------+-----------+----------+----------+----------+
|Communication Ser...|    EA|     146.95|08/06/2020|     86.94|03/20/2020|
|Communication Ser...|  TTWO|     177.52|08/04/2020|    100.15|03/20/2020|
|Communication Ser...|  VIAC|      41.16|01/07/2020|     11.19|03/23/2020|
|Communication Ser...|   DIS|      148.2|01/02/2020|     85.76|03/23/2020|
|Communication Ser...|     T|      37.03|01/22/2020|     25.39|03/23/2020|
+--------------------+------+-----------+----------+----------+----------+
only showing top 5 rows



In [117]:
consumerdiscretionarytable.show(5)

+--------------------+------+-----------+----------+----------+----------+
|              Sector|Symbol|Period_High| High_Date|Period_Low|  Low_Date|
+--------------------+------+-----------+----------+----------+----------+
|Consumer Discreti...|   LEN|      84.68|10/15/2020|     29.12|03/23/2020|
|Consumer Discreti...|   LEN|      84.68|10/16/2020|     29.12|03/23/2020|
|Consumer Discreti...|    UA|      19.27|01/03/2020|      6.89|04/02/2020|
|Consumer Discreti...|  ULTA|     301.34|02/20/2020|    128.52|03/18/2020|
|Consumer Discreti...|   HAS|     102.31|01/22/2020|     43.55|03/16/2020|
+--------------------+------+-----------+----------+----------+----------+
only showing top 5 rows



In [120]:
utilitiestable.show(5)

+---------+------+-----------+----------+----------+----------+
|   Sector|Symbol|Period_High| High_Date|Period_Low|  Low_Date|
+---------+------+-----------+----------+----------+----------+
|Utilities|   PNW|     100.56|02/21/2020|     61.07|03/23/2020|
|Utilities|  EVRG|      72.06|02/19/2020|     44.11|03/23/2020|
|Utilities|   CMS|      67.47|02/18/2020|     47.53|03/23/2020|
|Utilities|    SO|      68.59|01/30/2020|     42.23|03/23/2020|
|Utilities|   NEE|      77.54|11/11/2020|     44.94|03/23/2020|
+---------+------+-----------+----------+----------+----------+
only showing top 5 rows



In [125]:
financialstable.show()

+----------+------+-----------+----------+----------+----------+
|    Sector|Symbol|Period_High| High_Date|Period_Low|  Low_Date|
+----------+------+-----------+----------+----------+----------+
|Financials|  SPGI|     378.41|09/02/2020|    191.28|03/23/2020|
|Financials|    RF|      16.48|01/02/2020|      7.17|03/18/2020|
|Financials|   AXP|     134.99|02/19/2020|     67.99|03/23/2020|
|Financials|   PFG|      54.92|01/17/2020|     23.53|03/23/2020|
|Financials|   BLK|     670.62|11/11/2020|    323.31|03/23/2020|
|Financials|  TROW|      148.1|10/23/2020|     84.17|03/23/2020|
|Financials|    RE|     285.64|02/13/2020|    159.37|05/14/2020|
|Financials|   PRU|      92.62|01/17/2020|     37.84|03/23/2020|
|Financials|   ALL|     123.54|02/14/2020|      72.4|03/23/2020|
|Financials|    GL|      110.3|02/20/2020|      57.7|03/23/2020|
|Financials|  MKTX|     580.57|11/05/2020|    280.36|03/23/2020|
|Financials|   PNC|     154.39|01/09/2020|      78.0|03/23/2020|
|Financials|   JPM|     1

In [129]:
materialstable.show(5)

+---------+------+-----------+----------+----------+----------+
|   Sector|Symbol|Period_High| High_Date|Period_Low|  Low_Date|
+---------+------+-----------+----------+----------+----------+
|Materials|   AVY|     152.35|11/09/2020|     77.95|03/23/2020|
|Materials|   PPG|     142.82|11/10/2020|     71.46|03/23/2020|
|Materials|   EMN|       93.4|11/10/2020|      37.7|03/23/2020|
|Materials|   NUE|      53.44|01/02/2020|     27.56|03/23/2020|
|Materials|   NEM|       70.1|08/05/2020|     39.18|03/13/2020|
+---------+------+-----------+----------+----------+----------+
only showing top 5 rows



In [73]:
ind

[('Industrials',),
 ('Health Care',),
 ('Information Technology',),
 ('Communication Services',),
 ('Consumer Discretionary',),
 ('Utilities',),
 ('Financials',),
 ('Materials',),
 ('Real Estate',),
 ('Consumer Staples',),
 ('Energy',)]

In [24]:
health_care.coalesce(1).write.option("header","true").option("sep",",").mode("overwrite").csv("healthcare")

In [25]:
health_care.write.csv('health_care.csv')

In [12]:
new1.filter(new1.Sector=='Health Care').show()

NameError: name 'new1' is not defined

In [ ]:
new1=sectorjoin[(sectorjoin.Sector=='Health Care')].groupBy(['Symbol','Sector']).agg(fc.max('close').alias('_2020HIGH'))


In [ ]:
sectorjoin[(sectorjoin.Symbol=='RTX')].show()

In [ ]:
new1.show()

In [ ]:
new2=new1.join(sectorjoin,(new1.Symbol==sectorjoin.Symbol)).select(new1.Symbol,new1.Sector,sectorjoin.close,new1._2020HIGH,sectorjoin.date)


In [ ]:
new2.show(5)

In [26]:
connection = pymysql.connect(host='127.0.0.1',
                             user='root',
                             password='PApOEaPbJNOM62p1',
                             db='stockdb'
                            ,port=1234)

In [27]:
cur = connection.cursor()

In [202]:
sql_code=('DROP TABLE Industry_Health_Care')
cur.execute(sql_code)
test1 = cur.fetchall()
test1

()

In [149]:
health_caredf = healthcaretable.select("*").toPandas()
energydf = energytable.select("*").toPandas()
informationtechdf=informationtechtable.select("*").toPandas()
industrialsdf=industrialstable.select("*").toPandas()
communicationservicedf=communicationservicetable.select("*").toPandas()
consumerdiscretionarydf=consumerdiscretionarytable.select("*").toPandas()
utilitiesdf=utilitiestable.select("*").toPandas()
financialsdf=financialstable.select("*").toPandas()
materialsdf=materialstable.select("*").toPandas()
realestatedf=realestatetable.select("*").toPandas()
consumerstaplesdf=consumerstaplestable.select("*").toPandas()



In [193]:
health_caredf.dtypes

Sector          object
Symbol          object
Period_High    float64
High_Date       object
Period_Low     float64
Low_Date        object
DateH           object
DateL           object
dtype: object

In [191]:
health_caredf['DateH']= health_caredf['High_Date'].astype(str)
health_caredf['DateL']= health_caredf['Low_Date'].astype(str)
    

In [167]:
health_caredf.dtypes

Sector          object
Symbol          object
Period_High    float64
High_Date       object
Period_Low     float64
Low_Date        object
dtype: object

In [179]:
health_caredf.dtypes

Sector                 object
Symbol                 object
Period_High           float64
High_Date              object
Period_Low            float64
Low_Date               object
DateH          datetime64[ns]
DateL          datetime64[ns]
dtype: object

In [219]:
consumerdiscretionarytable.coalesce(1).write.option("header","true").option("sep",",").mode("overwrite").csv("consumerdiscre")

In [203]:
#for row in health_caredf.iterrows():
try:
    cur.execute("CREATE TABLE Industry_Health_Care (sector varchar(250),symbol varchar(250),periodhigh varchar(250),datehigh text,periodlow varchar(250),datelow text)")
    #health_caredf['DateH']= pd.to_datetime(health_caredf.High_Date)
    #health_caredf['DateL']= pd.to_datetime(health_caredf.Low_Date)
    for row in health_caredf.iterrows():
        idx = row[0]
        sector = row[1][0]
        symbol = row[1][1]
        high = row[1][2]
        dateh=row[1][6]
        low=row[1][4]
        datel=row[1][7]
        print(datel)
        cur.execute(f"INSERT INTO Industry_Health_Care (sector,symbol,periodhigh,datehigh,periodlow,datelow) VALUES ('{sector}','{symbol}', '{high}',{dateh},{low},{datel});")

except Exception as e:
    print(e)

03/16/2020
03/20/2020
03/16/2020
03/23/2020
03/23/2020
03/23/2020
04/03/2020
03/23/2020
03/23/2020
03/18/2020
10/30/2020
03/23/2020
03/12/2020
03/23/2020
03/23/2020
03/12/2020
03/23/2020
03/23/2020
03/18/2020
03/23/2020
03/23/2020
03/23/2020
03/23/2020
03/23/2020
03/23/2020
03/16/2020
03/18/2020
03/23/2020
03/23/2020
03/20/2020
03/20/2020
03/23/2020
03/23/2020
03/23/2020
03/23/2020
03/23/2020
03/12/2020
03/23/2020
03/23/2020
03/23/2020
03/23/2020
03/23/2020
03/23/2020
03/23/2020
03/23/2020
03/16/2020
03/23/2020
03/23/2020
01/30/2020


KeyboardInterrupt: 

In [30]:
result_pdf

,Symbol,Sector,Period_High,Period_Low
0,AMGN,Health Care,259.21,179.81
1,INCY,Health Care,109.69,63.18
2,REGN,Health Care,658.21,336.18
3,MCK,Health Care,183.44,115.13
4,ILMN,Health Care,400.74,209.20
...,...,...,...,...
57,XRAY,Health Care,60.28,31.66
58,WAT,Health Care,244.09,162.36
59,ABC,Health Care,110.08,73.26
60,DGX,Health Care,129.37,72.08
